# ECON 490: Good Regression Practice (14)


## Prerequisites: 
--- 
1. Importing data into Stata.
2. Creating new varables using `generate` and `replace`.
3. Identifying percentiles in the data using `summarize` and `return list`.
4. Running OLS regressions.


## Learning Outcomes:  
---
1. Identify and correct for outliers by trimming or winsorizing the dependent variable. 
2. Identify and correct for the problem of multicollinearity.
3. Identify and correct for the problem of heteroskedasticity. 
4. Identify and correct for the problem of non-linearity. 


## 14.1 Dealing with Outliers 

Imagine that we have constructed a dependent variable which contains the wage growth of individual workers and we see that some worker's wages increased more than 400%. We might wonder if this massive change is an coding error made by the statisticians that produced the dataset. Even without that type of error, we might worry that the wage growth of small number of observations are a driving the results we are obtaining. If this is the case, we would be producing an inaccurate analysis based on results that are not associated with the majority of our observations. 

The standard practice in these cases is to either winsorize or trim the subset of observations that are used in that regression. Both practices remove the outlier values in the dependent variable to allow us to produce a more accurate empirical analysis. 

<div class="alert alert-block alert-warning">
    
<b>Warning:</b> You should only consider fixing outliers when there is a clear reason to address this issue. Do not apply the tools below if the summary statistics in your data make sense to you in terms of abnormal values. For example, outliers might be a sign that your dependent and explanatory variables have a non-linear relationship. If that is the case you will want to consider including an interaction term that addresses that non-linearity. 
    
</div>

### 14.1.1 Winsorizing your Dependent Variable

Winsorizing is the process of limiting extreme values in the dependent variable. to reduce the effect of (possibly erroneous) outliers. It consists on replacing values below the $a$ percentile by that percentile value, and values above the $b$ percentile by that percentile. Consider the following example using our fake data set:


In [ ]:
clear* 

cd "/Users/marinaadshade/Documents/TELF Project/raw"
use fake_data, clear 

In [ ]:
su earnings, d

From the summary statistics above you can see that that the income earned by the individual at the 1st percentile is 2,037.23 and that the lowest earner in the data set earned is 8.88.  

We can also see that income earned by the individual at the 99th percentile is only of 540,525.90 and that the highest earner in the data earned over 190,000,000!

This table suggests to us that there are large outliers in our dependent variable.

We want to get rid of these outliers by winsorizing our data set. What that will mean is replacing the earnings of all observations below the 1st percentile by exactly the earnings of the individual at the 1st percentile. And replacing the earnings of all observations above the 99th percentile by exactly the earnings of the individual at the 99th percentile. 

Recall that we can see how Stata stored the information in the previously run `summarize` command by using the command `return list`.

In [ ]:
return list

To winsorize this data, we do the following:

1. Create a new variable with the same values as earnings that we will call it earnings_winsor (`gen earnings_winsor = earnings`). The reason why we can decide to store the winsorized version of the dependent variable as a different variable is just for organizational purposes - we don't want to overwrite the original data set. 
2. If earnings are smaller than 1st percentile, we will replace the values of earnings_winsor with the earnings of the individual at the 1st percentile (stored in Stata in r(p1)).
3. If earnings are larger than 1st percentile, we will replace the values of earnings_winsor with the earnings of the individual at the 99th percentile (stored in Stata in r(p99).


You can run these commands yourself below:

In [ ]:
gen earnings_winsor = earnings
replace earnings_winsor = r(p1) if earnings_winsor<r(p1)
replace earnings_winsor = r(p99) if earnings_winsor>r(p99)

Let's take a look at the summary statistics of the original earnings variable and the new variable that we have created:

In [ ]:
su earnings earnings_winsor

Now you will use this new dependent variable in your regression analysis. If the outliers were not creating problems, there will be no change in the results. If they were creating problems, those problems will now be fixed. 

Let's take a look at this by first running the regression from [Module 12]() with the original earning variable. 


In [ ]:
capture drop logearnings
gen logearnings = log(earnings)
regress logearnings age 

Now we will run this again, using the new winsorized earning variable. 

In [ ]:
capture drop logearnings_winsor
gen logearnings_winsor = log(earnings_winsor)
regress logearnings_winsor age 

Do you think that in this case the outliers were having a significant impact?

### 14.1.2 Trimming your Dependent Variable 

Trimming consists of replacing both values below the $a$ percentile and values above the $b$ percentile by a missing value. The idea is that when that variable equals a missing value it won't be used in the regression. Stata by design does not include observations where there are missing variables in the command `regress`.

Here are the commands for trimming your variable. 

In [ ]:
capture drop earnings_trim
gen earnings_trim = earnings
replace earnings_trim = . if earnings_trim<r(p1)
replace earnings_trim = . if earnings_trim>r(p99)

And here is the result of the regression with the new dependent variable. 

In [ ]:
capture drop logearnings_trim
gen logearnings_trim = log(earnings_trim)
regress logearnings_trim age 

## 14.2 Multicollinearity 

If two variables are linear combinations of one another they are multicollinear. Ultimately, Stata will not allow you to include two variables in a regression that are perfect linear combinations of one another, such as a constant, a dummy variable for male and a dummy for female (since female = 1 - male). If you try this yourself you will see that one of those variables will be dropped from the regression "because of collinearity".


In [ ]:
cap drop male
gen male = sex == "M"

cap drop female 
gen female = sex == "F"

In [ ]:
reg logearnings male female

Is this a problem? Not really. Multicollinearity is a sign that a variable is not adding any new information. Notice that with the constant term and a male dummy we can know the mean earnings of the females. In this case, the constant term is by construction the mean earnings of females, and the male dummy gives the earning premium is paid to male workers.

While there are some statistical tests for multicollinearity, nothing beats having the right intuition when running regression. If there is an obvious case where two variables contain basically the same information, you should avoid including both in the analysis. 

For instance, we might have an age variable that includes both years and months (e.g. if a baby is 1 year and 1 month old, then this age variable would be coded as 1 + 1/12 = 1.083). If we included this variable in a regression that also include an age variable that includes only years (e.g the baby's age would be codeds as 1) then we would have the problem of multicollinearity. Because they are not perfectly collinear, Stata might still produce some results but the coefficients on these two variables would be biased. 

## 14.3 Heteroskedasticity 

When we run a linear regression we essentially split the outcome into a (linear) part explained by observables and an error term:
$$ y_i = a + b x_i + e_i$$ 

The standard errors in our coefficients depend on $e_i^2$ (as you might remember from ECON 326). Heteroskedasticity refers to the case where the variance of this projection error depends on the observables $x_i$. For instance, the variance of wages tends to be higher for people who are university educated (there is some people with very high wages) whereas it is small for people who are non-university educated (they tend to be concentrated in smaller paying jobs).  Stata by defaults assumes that the variance does not depend on the observables, also known as homoskedasticity. It is safe to say that this is an incredibly restrictive assumption.

While there are tests for heteroskedasticity, the standard applied econometrician relies on including the option `robust` at the end of the `regress` command for the OLS regression. 

In [ ]:
cap drop logearnings
gen logearnings = log(earnings)
regress logearn age, robust

Best practices are simply to always use robust standard errors in your ECON 490 project.

## 14.4 Non-linearity

Our regression analysis so far assumes that the relationship between our independent and explanitory variables is linear. If this is not the case, and the relationship is non-linear, then we are getting inaccurate results with our analysis. 

Let's consider an example. We know that earnings increases with age, but what if economic theory predicts that the amount that earnings increases for each year of age when workers are younger is larger than the amount that earnings increases for each year of age when workers are older? Another way to say this is, what if we predict that earnings is increasing with age at a decreasing rate?

We could correct for this in our model by including a new interaction term that is simply age interacted with itself. You learned how to do this in [Module 13](). Let's include this in regression above remembering the age is a continuous variable 

In [ ]:
reg logearning c.age##c.age

There does seem to be some evidence in our regression results that this economic theory is correct since the coefficent on the interaction term is both negative and statistically significant.

<div class="alert alert-info">


**Note:** If there is a strong theoretical reason for believing that non-linearity exists Stata provides some tests for non-lineary. You can also created a scatterplot to see if you can observe a non-linear relationship in the data. We covered that approach in [Module 9]().
    
</div>




## 14.4 Wrap up 

While it is important to always follow best practices for regression analysis, checking and correcting for outliers, heteroskedaticy, multicollinearity and non-linearity can be more of an art than a science. If you need any guidance on whether or not you need to address these issues please be certian to speak with your instructor or your TA.